In [14]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import datetime

In [15]:
dataset_path = './download'

log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    labels="inferred",
    label_mode="categorical",
    seed=123,
    image_size=(150, 150),
    batch_size=50)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    labels="inferred",
    label_mode="categorical",
    seed=123,
    image_size=(200, 200),
    batch_size=75)

train_dataset = train_dataset.map(lambda x, y: (x/255, y))
validation_dataset = validation_dataset.map(lambda x, y: (x/255, y))

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

Found 9878 files belonging to 126 classes.
Using 7903 files for training.


In [17]:
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Conv2D, MaxPooling2D


base_model = MobileNetV3Small(input_shape=(150, 150, 3),
                              include_top=False,
                              weights='imagenet')


base_model.trainable = False


model = Sequential([
    Dense(126, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 5, 5, 576)        939120    
 l)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 32)          165920    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 1, 32)         0         
 2D)                                                             
                                                                 
 global_average_pooling2d_2   (None, 32)               0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                      

In [18]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
history = model.fit(
    train_dataset,
    epochs=15,
    validation_data=validation_dataset)

Epoch 1/15


In [ ]:
model.save('model3_rotation.h5')

In [ ]:
validation_dataset

In [ ]:
model.predict(validation_dataset)

27/27 [==============================] - 1s 21ms/step


array([[2.83778918e-05, 2.41348602e-07, 3.50015875e-06, ...,
        2.96170356e-06, 3.74000992e-05, 4.13633405e-10],
       [3.80243137e-07, 1.56013357e-07, 3.21421851e-07, ...,
        5.16722398e-09, 1.59183941e-07, 5.58022766e-05],
       [9.82785082e-07, 1.15401839e-04, 1.72984510e-05, ...,
        7.37674674e-03, 8.00510236e-07, 1.43062454e-04],
       ...,
       [1.56707669e-04, 3.86934630e-07, 1.31228023e-06, ...,
        7.93157378e-05, 4.44753331e-11, 5.74968115e-04],
       [1.30780293e-07, 3.51248673e-06, 3.79540815e-10, ...,
        8.10225629e-06, 1.58236535e-08, 2.73584053e-08],
       [1.20641360e-07, 6.30994748e-07, 4.91274204e-06, ...,
        1.59401097e-03, 6.99412250e-10, 1.14235235e-07]], dtype=float32)